In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import pandas as pd
import numba as nb
from numba.experimental import jitclass

from neo_backtesting import backtest_online, SingleFutureSimulator
from strategy_example.bolling import BollingPosMgtStrategy, boll_factor_cross_timeframe

In [3]:
# 回测起始日
START_DATE = '20180301'

# 布林带周期与宽度
N = 100  # 100 小时
B = 2

# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万美元
MAX_LEVERAGE = 1.5  # 最大使用 1.5 倍杠杆
MAX_LOSS = 0.05  # 最大开仓风险 5%
STOP_PCT = 0.05  # 移动止损 5%
STOP_CLOSE_PCT = 0.2  # 每次止损平仓 20%
FACE_VALUE = 0.01  # 合约面值 0.01 ETH
COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 3

In [4]:
df_1h = pd.read_feather('../candle_1h.fea')
df_1m = pd.read_feather('../candle_1m.fea')

df = boll_factor_cross_timeframe(df_1m, df_1h, N, B)

df = df[df['candle_end_time'] >= START_DATE].reset_index(drop=True)

In [6]:
%%time

simulator = SingleFutureSimulator(
    init_capital=INIT_CAPITAL, 
    face_value=FACE_VALUE, 
    comm_rate=COMM_RATE, 
    liqui_rate=LIQUI_RATE, 
    init_pos=0)

strategy = BollingPosMgtStrategy(
    max_leverage=MAX_LEVERAGE, 
    max_loss=MAX_LOSS, 
    face_value=FACE_VALUE, 
    stop_pct=STOP_PCT, 
    stop_close_pct=STOP_CLOSE_PCT)

backtest_online(
    df, 
    simulator=simulator,
    strategy=strategy,
    factor_columns = ['upper', 'lower', 'median'])
print(df[['candle_begin_time', 'pos', 'equity']].tail().to_markdown(), '\n')

|         | candle_begin_time   |     pos |      equity |
|--------:|:--------------------|--------:|------------:|
| 2249268 | 2022-06-13 23:55:00 | -714816 | 5.22373e+07 |
| 2249269 | 2022-06-13 23:56:00 | -714816 | 5.22318e+07 |
| 2249270 | 2022-06-13 23:57:00 | -714816 | 5.22399e+07 |
| 2249271 | 2022-06-13 23:58:00 | -714816 | 5.22389e+07 |
| 2249272 | 2022-06-13 23:59:00 | -714816 | 5.22121e+07 | 

CPU times: user 196 ms, sys: 84.8 ms, total: 281 ms
Wall time: 279 ms
